In [1]:
#!pip install pymupdf
#!pip install langchain
#!pip install lancedb
#!pip install -U langchain_ollama

In [2]:
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import LanceDB
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM

In [4]:
import lancedb

/home/u/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
doc = fitz.open("DeepSeek.pdf")

In [19]:
texts, metadatas = [], []


In [20]:
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    texts.append(page.get_text())
    metadatas.append(doc.metadata)

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document=text_splitter.create_documents(texts,metadatas=metadatas)

In [22]:
embeddings = OllamaEmbeddings(model="deepseek-r1:14b")

In [23]:
db = lancedb.connect("./lancedb")

In [24]:
vectorstore = LanceDB.from_documents(document, embeddings, connection=db)

In [25]:
llm = OllamaLLM(model="deepseek-r1:14b")

In [26]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [27]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [28]:
query = "What are the main topics discussed in the documents?"

In [29]:
result = qa.invoke(query)


In [17]:
print(result)

{'query': 'What are the main topics discussed in the documents?', 'result': '<think>\n嗯，让我仔细看看这个问题。用户问的是：“What are the main topics discussed in the documents?” 翻译过来就是“文件中讨论了哪些主要主题？”。根据提供的上下文信息，我需要找出相关的内容来回答。\n\n首先，看第一个部分是关于Deepseek的介绍，包括它是什么、能做什么以及如何使用。这些内容显然是在讨论Deepseek的功能和使用方法，所以这是一个主要的主题。\n\n接下来的部分提到了TASTE框架，里面详细列出了任务（Task）、目标受众（Audience）、结构（Structure）、语气（Tone）和示例（Example）。然后是ALIGN框架，包括目标、难度级别、输入、指导原则和新颖性。这些都是关于如何有效使用AI的策略，所以这也是一个主要的主题。\n\n然后有一个提示词框架，涉及逻辑锚定和思维引导，以及具体的示例。这部分可能涉及到如何设计有效的提示语来优化AI的表现，这也是讨论的内容之一。\n\n接下来的信息传递部分强调了清晰和精准的信息框架设计，提到了五种提示语设计技巧：信息层次、表达简洁、结构严密、受众针对性和记忆点突出。这些都是关于如何更好地传递信息的策略，所以也是一个主要主题。\n\n最后的应用示例部分详细说明了如何为产品创作文案，包括核心信息、功能详解、目标受众说明等。这表明讨论还包括如何在商业文案中应用这些框架和技巧，所以这也是一个主题。\n\n综上所述，文件讨论的主要主题围绕Deepseek的功能与使用方法，利用TASTE和ALIGN框架优化AI的使用，设计有效的提示词来提升信息传递的效果，以及在商业文案中的实际应用策略。\n</think>\n\nThe main topics discussed in the documents are:\n\n1. **Introduction to DeepSeek**: What it is, what it can do, and how to use it.\n\n2. **Framework for Effective AI Usage**:\n   - TASTE Fram